In [1]:
#!unzip jigsaw-unintended-bias-train.csv.zip

In [2]:
#!pip install transformers
#!pip install transformers[sentencepiece]

In [3]:
#language_supported=['ar_AR', 'de_DE', 'en_XX', 'es_XX', 'fi_FI', 'fr_XX', 'hi_IN', 'it_IT', 'ja_XX', 'ko_KR']
                    
#language_supported=['nl_XX', 'ro_RO', 'ru_RU', 'tr_TR', 'vi_VN', 'zh_CN', 'bn_IN', 'he_IL', 'hr_HR', 'id_ID']
                    
language_supported=['tl_XX','uk_UA']

## , 'ur_PK'

len(language_supported)


2

In [4]:
import pandas as pd 

df=pd.read_csv("jigsaw-unintended-bias-train.csv")

df.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxicity', 'obscene',
       'identity_attack', 'insult', 'threat', 'asian', 'atheist', 'bisexual',
       'black', 'buddhist', 'christian', 'female', 'heterosexual', 'hindu',
       'homosexual_gay_or_lesbian', 'intellectual_or_learning_disability',
       'jewish', 'latino', 'male', 'muslim', 'other_disability',
       'other_gender', 'other_race_or_ethnicity', 'other_religion',
       'other_sexual_orientation', 'physical_disability',
       'psychiatric_or_mental_illness', 'transgender', 'white', 'created_date',
       'publication_id', 'parent_id', 'article_id', 'rating', 'funny', 'wow',
       'sad', 'likes', 'disagree', 'sexual_explicit',
       'identity_annotator_count', 'toxicity_annotator_count'],
      dtype='object')

In [5]:
df=df[df["rating"]=="approved"]
df=df[['id','comment_text', 'toxic', 'severe_toxicity', 'obscene',
       'identity_attack', 'insult', 'threat', 'sexual_explicit', 'rating']]

In [6]:
df["length"]=df['comment_text'].apply(lambda x: len(x.split()))
df=df[df["length"]<=20]
df = df.drop_duplicates(subset=['comment_text'])

In [7]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import torch

# Check if a GPU is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt").to(device)

tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

In [8]:
!nvidia-smi

Sun Dec 17 06:25:33 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       On  | 00000000:00:1E.0 Off |                    0 |
| N/A   29C    P0              30W /  70W |   2445MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# Set the source language (assuming it's English)
tokenizer.src_lang = "en_XX"

# Define the batch size
batch_size = 20

for lang in language_supported:
    print(lang)
    
    # Create an empty DataFrame for the translated data
    translated_data_list = []
    sampled_dataframe = df.sample(n=150000, random_state=42)  # You can use any random_state value for reproducibility

    # Iterate over batches of input sentences
    for batch_start in range(0, len(sampled_dataframe), batch_size):
        batch_end = batch_start + batch_size
        batch_comments = sampled_dataframe['comment_text'].iloc[batch_start:batch_end].tolist()

        # Encode the batch
        encoded_batch = tokenizer(batch_comments, return_tensors="pt", padding=True, truncation=True).to(device)
        
        # Generate translations
        generated_tokens = model.generate(
            **encoded_batch,
            forced_bos_token_id=tokenizer.lang_code_to_id[lang],
            max_length=256  # Adjust this based on your maximum sequence length
        )
        
        # Decode and collect translated texts
        translated_texts = tokenizer.batch_decode(generated_tokens.cpu(), skip_special_tokens=True)
        
        # Create a DataFrame for the batch
        batch_data = [{"id": sampled_dataframe['id'].iloc[batch_start + index], f'comment_text_{lang}': text}
                      for index, text in enumerate(translated_texts)]
        
        # Append the batch data to the translated_data_list
        translated_data_list.extend(batch_data)
    
    # Create a DataFrame for all translated data
    df_new = pd.DataFrame(translated_data_list)
    
    # Save the translated data to a CSV file
    df_new.to_csv(f"translated_data_{lang}.csv", index=False)

tl_XX


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
lang

In [ ]:
len(translated_data_list)

In [ ]:
lang="ta_IN"
# Create a DataFrame for all translated data
df_new = pd.DataFrame(translated_data_list)

# Save the translated data to a CSV file
df_new.to_csv(f"translated_data_{lang}.csv", index=False)

In [ ]:
df_new